# Slide Inference

## Import Packages

In [1]:
import sys
sys.path.insert(0, '../segmentation')
from evaluation_helper import *
from slide.slide_container import SlideContainer
import h5py
import time

## Define model input parameters

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
segmentation_patch_size = 512
classification_patch_size = 1024
batch_size = 8
segmentation_level = 2
classification_level = 0

## Define paths of slide folder and annotation JSON

In [3]:
target_folder = Path("../catch")
annotation_file = "../CATCH.json"

## Define dicts to match annotation classes to classes used for training

In [4]:
segmentation_label_dict = {'Bg': 0, 'Bone': -1, 'Cartilage': -1, 'Dermis': 1, 'Epidermis': 2, 'Subcutis': 3,
                           'Inflamm/Necrosis': 4, 'Melanoma': 5, 'Plasmacytoma': 5, 'Mast Cell Tumor': 5, 'PNST': 5,
                           'SCC': 5, 'Trichoblastoma': 5, 'Histiocytoma': 5}

classification_label_dict = {'Bg': -1, 'Bone': -1, 'Cartilage': -1, 'Dermis': 0, 'Epidermis': 0, 'Subcutis': 0,
                             'Inflamm/Necrosis': 0, 'Melanoma': 1, 'Plasmacytoma': 2, 'Mast Cell Tumor': 3, 'PNST': 4,
                             'SCC': 5, 'Trichoblastoma': 6, 'Histiocytoma': 7}

## Load slide in custom SlideContainer format

In [5]:
segmentation_container = SlideContainer(Path(str(target_folder) + "/PNST/PNST_09_1.svs"), annotation_file, segmentation_level,segmentation_patch_size, 
                                             segmentation_patch_size, dataset_type="segmentation", label_dict=segmentation_label_dict)
classification_container = SlideContainer(Path(str(target_folder) + "/PNST/PNST_09_1.svs"), annotation_file, classification_level, classification_patch_size, 
                                               classification_patch_size,dataset_type="classification",label_dict=classification_label_dict)

## Load learners

In [6]:
segmentation_learner = load_learner(Path("../models"), "UNet_resnet18_512_2_sdata.pkl")
classification_learner = load_learner(Path("../models"), "efficientnet_1024_0_sdata.pkl")

## Slide inference

In [7]:
store = h5py.File("{}.hdf5".format(Path(segmentation_container.slide._filename).stem), "a")
start_time = time.time()
classification_indices = segmentation_inference(segmentation_container.slide, store, segmentation_patch_size, segmentation_level, batch_size, segmentation_learner, overlap_factor=0.5) 
print("Segmentation: --- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
classification_inference(classification_container.slide, store, classification_patch_size, classification_level, batch_size, classification_learner, classification_indices) 
print("Classification: --- %s seconds ---" % (time.time() - start_time))

Processing PNST_09_1:   0%|                                                                    | 0/737 [00:00<?, ?it/s]

Segmentation: --- 79.74933767318726 seconds ---


Processing PNST_09_1: 100%|██████████████████████████████████████████████████████████| 737/737 [15:52<00:00,  1.29s/it]

Classification: --- 952.2370300292969 seconds ---


## Read inference masks

In [8]:
store = h5py.File("{}.hdf5".format(Path(segmentation_container.slide._filename).stem), "r")

## Segmentation Jaccard score

In [9]:
cm_matrix_s = segmentation_cm_matrix(segmentation_container, store, segmentation_learner.data.c)
slide_jaccard_score(cm_matrix_s, segmentation_learner.data.classes)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [02:30<00:00, 11.57s/it]

IoUs:  {'Bg': 0.9798, 'Dermis': 0.5998, 'Epidermis': 0.5536, 'Subcutis': 0.2903, 'Inflamm/Necrosis': 0.2388, 'Tumor': 0.932} Mean:  0.5990551299606123
Frequency-Weighted IoU:  0.9193


## Classification evaluation

In [10]:
cm_matrix_c = classification_cm_matrix(classification_container, store, classification_learner.data.c)
slide_tumor_recall(cm_matrix_c, classification_learner.data.classes)
store.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [06:10<00:00,  3.78s/it]

Slide Label:  PNST
Slide Classification:  PNST
Tumor Recall:  0.9692
